In [1]:
%run C:/__PYP/__COMMON/ugenejupyterlib/firstcell.ipynb
%run C:/__PYP/__COMMON/ugenejupyterlib/datacell.ipynb

@
        # platform.node()='MH507192' 
        # datetime.now()=datetime.datetime(2020, 10, 20, 7, 49, 12, 608140) 
        # sys.version='3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]' 
        


In [2]:
V=\
"""
Foos
"""
import yaml
from pathlib import Path
import re, os

def foo_initMeta():

    with open("0_init.yaml", 'r') as stream:
        try:
            yamldct = (yaml.load(stream, Loader=yaml.FullLoader))
        except yaml.YAMLError as e:
            print(e)
            return 0


        # pickOutput's file name and file path will be calculated in the foo_listFilesToRead()


    # for data2excel and plotting:
    pickleToRead = yamldct["filename_inputPickle"]
    yamldct['filepath_inputPickle'] = Path(yamldct['pwd'], 'pickles', f'{pickleToRead}')
    yamldct['dirpath_outputs'] = Path(yamldct['pwd'], 'outputs/',)
    yamldct['filepath_jpeg'] = Path(yamldct['pwd'], 'outputs/', f'{pickleToRead}.jpg')
    yamldct['filepath_excel'] = Path(yamldct['pwd'], 'outputs/', f'{pickleToRead}.xlsx')
    yamldct['filepath_csv'] = Path(yamldct['pwd'], 'outputs/', f'{pickleToRead}.csv')


    # for raw txt files reading:
    rex_partfilePattern = re.compile(
        r'''
        (?P<subject>.*)
        _part(?P<part>\d+)
        \.txt$
        ''',
        re.X,
    )

    rex_replacePartNum = re.compile(
        r'''
        _part\d+ \.txt
        ''',
        re.X,
    )

    if yamldct['yes_readEntireFolder']:
        folderToRead = yamldct["all txts in this folder will be read"]
        yamldct['dirpath_txts'] = Path(yamldct['pwd'], 'raws', f'{folderToRead}')

        yamldct['filename_outputPickle'] = folderToRead



    else:
        yamldct['dirpath_txts'] = Path(yamldct['pwd'], 'raws')

        oneofthepartfile = yamldct["one of the part files in /raws"]
        match = rex_partfilePattern.match(oneofthepartfile)
        yamldct['filename_outputPickle'] = match.group('subject')

    yamldct['filepath_outputPickle'] = Path(yamldct['pwd'], 'pickles/', f"{yamldct['filename_outputPickle']}")

    yamldct['filepath_txt_stub'] = Path(yamldct['dirpath_txts'], yamldct['filename_outputPickle'])
    # this is the partial fullpath of a partfile that doesn't have _part# attached
    # it is used to generate a list of partfiles
    return yamldct


def listFilesToRead():
    '''
    read 0_init.yaml and inferr all paths filenames
    it is called by all utils
    '''

    meta_ = foo_initMeta()

    # create folders for rawinput, pickled data, and output
    #   if they don't exist yet.
    folders = dict()
    yes_allNecessaryFoldersExist = 1
    for x in ['raws', 'pickles', 'outputs']:
        foldername_toBeCreated = x
        p = os.path.join(meta_['pwd'], foldername_toBeCreated)
        folders[x] = p
        if not os.path.exists(p):
            os.makedirs(p)
            yes_allNecessaryFoldersExist = 0

    if not yes_allNecessaryFoldersExist:
        import logging
        logging.warning(
            "first time run, 'raws', 'pickles', 'outputs' created. Run Again!"
        )
        exit(0)


    l_return = [] # list of filepaths to return
    subject = '' # the pickleoutput filename



    if meta_['yes_readEntireFolder']:
        # rename files to be partfiles anyways according to modified time

        import logging
        logging.warning(
            'reading entire folder!'
        )

        dirpath_txts = meta_['dirpath_txts']
        subject = meta_['filename_outputPickle']

        from collections import OrderedDict
        dct_tobeSorted :dict = {}

        for x in os.listdir(dirpath_txts):
            if '.txt' in x and not(subject+'_part' in x):
                xpath = os.path.join(dirpath_txts, x)
                if os.path.isfile(xpath) :
                    dct_tobeSorted[os.path.getmtime(xpath)] = xpath

        dct_sorted = OrderedDict(sorted(dct_tobeSorted.items()))

        partnumber = 4
        for x in dct_sorted:

            newfilepath = os.path.join(dirpath_txts, f'{subject}_part{partnumber:02}.txt')
            logging.warning(
                f'renamed {dct_sorted[x]} into {newfilepath}'
            )
            os.rename(
                dct_sorted[x],
                newfilepath,
            )
            partnumber += 4

    '''
    check how many parts file are there to read
    parts number does not have to be continuous
    '''
    filepath_stub = meta_['filepath_txt_stub']

    for i in range(0, 100):  # part number must be in the range() here

        # we are not using the os.listdir() or os.walk() here
        # because we want to make sure that the part files are read
        # in the incrementing sequence
        replstr = f'_part{i:02}.txt'

        xfilepath = filepath_stub.with_name(filepath_stub.name + replstr)
        if os.path.isfile(xfilepath):
            l_return.append(xfilepath)
    meta_['l_filepaths'] = l_return
    return meta_


In [3]:

######################################################################
# [USER Choices]
######################################################################

yes_filter_fault_data = 0
yes_time_constrain = 0
histPointsStyle = 'wk'
    # LS or KT == 'normal', LM =='lm'
######################################################################

import pickle
meta = listFilesToRead()
l_filepathToRead = meta['l_filepaths']
subject = meta['filename_outputPickle']
filepath_pickle = meta['filepath_outputPickle']


l_dfs = []  # list of dfs, contains all individual dfs with their own timestamp

for xfilepath in l_filepathToRead:
    # build up individual dfs from df_raw
    c=r"""
    the format of the raw file (tab separated txt file):
    tagname	timestamp	value
    \\KTGS_UCMS\U12_HDCV_AC_PMP_RUN_AUTO.VALUE	7/24/2017 09:16:23.875	1
    \\KTGS_UCMS\U12_HDCV_AC_PMP_RUN_AUTO.VALUE	7/24/2017 09:19:45.703	0
    \\KTGS_UCMS\U12_HDCV_DC_PMP_RUN_AUTO.VALUE	7/24/2017 09:16:52.859	1
    \\KTGS_UCMS\U12_HDCV_DC_PMP_RUN_AUTO.VALUE	7/24/2017 09:19:45.703	0
    \\KTGS_UCMS\U12_HDCV_BKUP_TEST.VALUE	7/24/2017 09:16:22.875	0
    \\KTGS_UCMS\U12_HDCV_BKUP_TEST.VALUE	7/24/2017 09:22:22.875	1
    \\KTGS_UCMS\U12_HDCV_WTR_LEV_VAL.VALUE	7/24/2017 09:11:11.171	12
    \\KTGS_UCMS\U12_HDCV_WTR_LEV_VAL.VALUE	7/24/2017 09:11:15.156	13
    """


    print('reading: ', xfilepath, ' ...')
    df_raw = pd.read_csv(xfilepath, sep='\t', thousands=',', parse_dates=[2])  # header=None,
    # the kargs 'thousands' is very useful if the number contains ','

    df_raw = df_raw.dropna(axis='columns', how='all')  # this is necessary, the first column is empty
    # duplicated rows don't seem to matter.
    # @ df_raw.columns = ['tagname', 'timestamp', 'value']

    uniqueTags = df_raw['tagname'].unique()

    list_columnNames = list(df_raw.columns.values)
    for i in range(len(uniqueTags)):

        # filter out one tag's analogue data
        # @idf contains 3 columns 'tagname', 'timestamp', 'value'
        idf = df_raw[
            df_raw['tagname'] == uniqueTags[i]
        ]

        
        c='''
        ######################################################################
        [STEP optional]
            parsing the historian points format, to extract point name
        ######################################################################
        '''        
        if histPointsStyle=='normal':
            # create the new tagname that is rid of fluffs in:
            #    '\\LSGS_UCMS\U3.GEN_GDBRG1_TEMP.VAL.VALUE'
            #    '\\LSGS_UCMS\U6.STTR_WDG_avgtemp.VALUE'
            tempTagname = uniqueTags[i].split('\\')[-1]
            tempTagname = re.search(r'(?P<keep>[\w\.]+?)(?=[_\.]VAL(UE)?)', tempTagname, re.I).group('keep')
            
        if histPointsStyle=='wk':        
            # WK.U3.GOV_PILOTVALVE_SIG # no val or value at the end
            tempTagname = uniqueTags[i][3:]
#             tempTagname = re.search(r'(?P<keep>[\w\.]+?)(?=[_\.]VAL(UE)?)', tempTagname, re.I).group('keep')


        # change the analogue-data-containing column's column name to be the cim point name
        list_columnNames[-1] = tempTagname
        idf.columns = list_columnNames
        idf = idf.drop('tagname', axis=1)
        idf['timestamp'] = pd.to_datetime(idf['timestamp']) # convert into numpy's proprietary datetime64 format

        c='''
        ######################################################################
        [STEP optional]
            specify start time and end time, if you want to truncate time
        ######################################################################
        '''

        if yes_time_constrain:

            ts_start_ = np.datetime64(datetime.datetime(2017, 2, 16, 3, 30, 0))
            ts_end_ = np.datetime64(datetime.datetime(2017, 2, 17, 0, 0, 0))

            idf = idf[
                (idf['timestamp'] > ts_start_) & (idf['timestamp'] < ts_end_)
            ]



        c='''
        ######################################################################
        [STEP optional]
            filtering out fault data like int being 32767, in ucms, that means
            fault
        ######################################################################
        '''
        if yes_filter_fault_data:
            idf.loc[
                (idf[tempTagname] > 200) | (idf[tempTagname] < -200),
                tempTagname
            ] = np.nan

        idf = idf.drop_duplicates(subset='timestamp', keep='first')
        # When joining dfs side by side (instead of joining dfs on top of each other)
        #   we can not have duplicated index.
        # [?] there are duplicated timestemp within an idf? anyways, this shouldn't cause any harm
        # [todo] investigate this syntax:
        #	df3 = df3[~df3.index.duplicated(keep='first')]
        #	this solves the problem of dropping index, while the index column doesn't have a name
        # [todo] concat by join='inner' does NOT get rid of repeated index, unless the entire row is the same

        idf = idf.set_index(['timestamp'])
        # @ idf ==> index=timestamp column0=U3.GEN_CUR_C

        flag0 = 0
        for i, xdf in enumerate(l_dfs):
            if idf.columns == xdf.columns:
                print('found existing columns')
                l_dfs[i] = pd.concat([xdf, idf], join = 'outer', axis=0)
                flag0 = 1

        if not flag0:
            l_dfs.append(idf)

print(f'generating meta file ... {meta["filename_outputPickle"]}'+'.txt')
with open(
        str(meta['filepath_outputPickle']) + '.txt',
        'w',
) as output_file:
    for idx, item in enumerate(l_dfs):
        tstr = '{:>2}: {:<22} => length: {} \n'.format(idx, item.columns[0], item.shape[0])
        output_file.write(tstr)
        print(tstr, end='')

print(f"pickling data ... {meta['filepath_outputPickle']}")
with open(
    meta['filepath_outputPickle'],
    'wb',
) as output_file:
    pickle.dump(l_dfs, output_file, protocol=pickle.HIGHEST_PROTOCOL)
    # VERY IMPORTANT to have the 'protocol' keyword argument, otherwise this won't work


reading:  data\raws\data\data_part04.txt  ...
reading:  data\raws\data\data_part08.txt  ...
reading:  data\raws\data\data_part12.txt  ...
generating meta file ... data.txt


41

 0: U3.GOV_WGATE_POSN      => length: 2 


43

 1: U3.GOV_PILOTVALVE_SIG  => length: 183 


...

 9: U2.GOV_PILOTVALVE_SIG  => length: 447 


42

10: U2.GOV_SP_IND          => length: 56 


42

11: U2.GEN_FREQ.VAL        => length: 51 
pickling data ... data\pickles\data


In [4]:
import pickle
import pandas as pd
import xlsxwriter


######################################################################
# [USER Choices]
######################################################################
yes_fillnan = 0
yes_outputXlwings = 0
yes_outputExcel = 1
yes_outputCSV = 0
######################################################################


'''
join list of dfs together, and use xlwings to export them
'''

yaml_ = foo_initMeta()

filepath = yaml_['filepath_inputPickle']

print(f'reading pickle file ... \n ==> {filepath}')
with open(
        filepath,
        'rb',
) as input_file:
    l_dfs = pickle.load(input_file)

# with open(r'input/data0', 'rb') as input_file:
#     dict_aux = pickle.load(input_file)
#
# if not dict_aux:
#     dict_aux = {'subject': 'result'}

df0 = pd.concat(l_dfs, axis=1, join='outer')
    # amazingly pandas' outer join can take care of unalinged timestamp merging
    # as long as timestamp is the index.



######################################################################
# [USER Choice] fill nan
######################################################################


if yes_fillnan:
    df0 = df0.fillna(
        method='ffill', # forward fill, meaning, fill with Previous data
        axis=0, # doc says "index", but it really means "along the column"
    )
    df0 = df0.fillna(
        method='bfill', # what if the first data is NaN? well fill backward
        axis=0,
    )



######################################################################
# [USER Choice] xlwings output
######################################################################

if yes_outputXlwings:
    import importlib.util
    spec = importlib.util.spec_from_file_location('asdf', r'C:\__PYP\__COMMON\foo_df2xw.py')
    myModuleName0_foo_df2xw = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(myModuleName0_foo_df2xw)

    xwsheet = myModuleName0_foo_df2xw.foo_df2xw(df0)

    xwsheet.range('A:A').number_format = 'yyyy-mm-dd hh:mm:ss'
    xwsheet.range('B:ZZ').number_format = '00.00'


######################################################################
# [USER Choice] xlsxwriter output
######################################################################

if yes_outputExcel:
    filepath_excel = yaml_['filepath_excel']
    # writer = pd.ExcelWriter(filepath_excel, engine='xlsxwriter')

    #
    # workbook = xlsxwriter.Workbook(filepath_excel, {'default_date_format':
    #                                                   'dd/mm/yy'})

    print(f'saving excel file ... \n ==> {filepath_excel}')

    writer = pd.ExcelWriter(
        filepath_excel,
        engine='xlsxwriter',
    )

    # [v] very necessary step to disabble pd's header format
    #    for some reason, this header format is applied at the end of xlsxwriter.save() operation
    #    so any attempts to apply header format yourself is futile

    try:
        import pandas.io.formats.excel
        pandas.io.formats.excel.header_style = None
    except Exception as e:
        print('attempting disable pandas default header format ==>', e)

    df0.to_excel(
        excel_writer=writer,
        index=True,
        sheet_name='raw',
        na_rep="--"
    )

    workbook :xlsxwriter.Workbook = writer.book
    print('reformatting...', workbook)
    worksheet = writer.sheets['raw']

    cellformat0 = workbook.add_format(
        {
            'num_format':'0.00',
            'font_size':12,
            'align':'right',
            'shrink':True,
        }
    )

    worksheet.set_column(0, 0, 20) # column 0 to 0, width of 20
    # [!!!] It is IMPOSSIBLE to format date/datetime column data using set_column()
    # it is impossible to format the header row.
    # xlsx writer is powerless to format anything preformatted by pandas
    # xlsx can still adjust width and height of columns and rows
    worksheet.set_column(1, 33, 8, cellformat0)

   #--------------------------------------------------------------------------
    # [v] format header rows

    comment='''
    # [philosophical] can not do accumulative formatting in xlsxwriter, that's why
    after column formatting, header format needs to be changed

    unlike in excel, you can apply accumulative formatting to a cell, ie. apply
    row format bold and column format fontcolor overlaps at a cell. that cell will 
    be bold and fontcolor. xlsxwriter can not do that.

    mostly because every combo of format you want must have an underlying xlsxwriter
    format object. Excel does this in the background so accumulative formatting
    is possible.
    '''

    # # [!!!] according to xlsxWriter doc: Pandas writes the dataframe header with a hardcoded cell
    # # format. Since it is a cell format it cannot be overridden using set_row().

    headerformatdict = {'align':'left', 'valign':'bottom', 'rotation':75, 'right': True, 'bold':True}
    headerformat = workbook.add_format(headerformatdict)
    for i, headername in enumerate(df0.columns.values):
        successful = worksheet.write(0,i+1, headername, headerformat)
    successful = worksheet.freeze_panes(1, 1)

    writer.save()


######################################################################
# [USER Choice] xlsxwriter or csv output
######################################################################

if yes_outputCSV:
    df0.to_csv(yaml_['filepath_csv'], sep='\t', encoding='utf-8')


'\njoin list of dfs together, and use xlwings to export them\n'

reading pickle file ... 
 ==> data\pickles\data
saving excel file ... 
 ==> data\outputs\data.xlsx
reformatting... <xlsxwriter.workbook.Workbook object at 0x000002B827712B20>


0

0